In [8]:
import pandas as pd
import numpy as np
import copy as cp
from scipy import stats
from math import floor
import matplotlib.pyplot as plt
import csv
from time import time
import io
import copy

In [9]:
anchor = """<a name="%s"></a><br/><br/><br/>\n"""

player_map = {
  'X': '✖️',
  'O': '⭕',
  '_': '⬜'
}

play_again = "[Care to play again?](#root)"
draw_message = "Guess we'll call it a draw"
win_message = "%s wins!!!"

header = """
<a name="top"></a>
I challenge you to a game of Tic-Tac-Toe. 
Give this page a few seconds to load all the emojis and then click the topmost box to start playing.

Xs go first, click one of the nine boxes to make your move.

_If for some reason the buttons are not clickable, try reloading the page. Sometimes GitHub can't handle all the emojis._

[Take me to the bottom](#bottom)

---
"""

footer = """

---

<a name="bottom"></a>
[Take me to the top](#top)
"""

In [10]:
class State:
    def __init__(self, moves):
        self.to_move = 'X'
        self.utility = 0
        self.board = {}
        self.moves = cp.copy(moves)
        
class TicTacToe:
    def __init__(self, nrow=3, ncol=3, nwin=3, nexp=0):
        self.nrow = nrow
        self.ncol = ncol
        self.nwin = nwin
        self.nexp = nexp
        self.state = State([(r, c) for r in range(1, nrow+1) for c in range(1, ncol+1)])        

    def result(self, move, state):
        if move not in state.moves:
            return state
        new_state = cp.deepcopy(state)
        new_state.utility = self.compute_utility(move, state)
        new_state.board[move] = state.to_move
        new_state.moves.remove(move)
        new_state.to_move = ('O' if state.to_move == 'X' else 'X')
        return new_state
        
    def compute_utility(self, move, state):
        row, col = move
        player = state.to_move
        
        board = cp.deepcopy(state.board)
        board[move] = player

        in_a_row = 0
        for c in range(1,self.ncol+1):
            in_a_row += board.get((row,c))==player
        in_a_col = 0
        for r in range(1,self.nrow+1):
            in_a_col += board.get((r,col))==player
        in_a_diag1 = 0
        for r in range(row,0,-1):
            in_a_diag1 += board.get((r,col-(row-r)))==player
        for r in range(row+1,self.nrow+1):
            in_a_diag1 += board.get((r,col-(row-r)))==player
        in_a_diag2 = 0
        for r in range(row,0,-1):
            in_a_diag2 += board.get((r,col+(row-r)))==player
        for r in range(row+1,self.nrow+1):
            in_a_diag2 += board.get((r,col+(row-r)))==player
        
        if self.nwin in [in_a_row, in_a_col, in_a_diag1, in_a_diag2]:
            return 1 if player=='X' else -1
        else:
            return 0

    def game_over(self, state):
        return state.utility!=0 or len(state.moves)==0

    def utility(self, state, player):
        return state.utility if player=='X' else -state.utility

    def move(self, move):
        self.state = self.result(move, self.state)

def alphabeta_search(game):
    player = game.state.to_move
    
    def max_value(state, alpha=-float('inf'), beta=float('inf')):
        if game.game_over(state):
            return game.utility(state, player), None
        
        bestVal = -float('inf')
        bestMove = None
        
        for move in state.moves:
            value, _ = min_value(game.result(move, state), alpha, beta)
            if value > bestVal:
                bestMove = move
            bestVal = max(bestVal, value)
            alpha = max(alpha, bestVal)
            if beta <= alpha:
                break
        return bestVal, bestMove

    def min_value(state, alpha=-float('inf'), beta=float('inf')):
        if game.game_over(state):
            return game.utility(state, player), None
        
        bestVal = float('inf')
        bestMove = None
        
        for move in state.moves:
            value, _ = max_value(game.result(move, state), alpha, beta)
            if value < bestVal:
                bestMove = move
            bestVal = min(bestVal, value)
            beta = min(beta, bestVal)
            if beta <= alpha:
                break
        return bestVal, bestMove
    
    _, move = max_value(game.state)
    return move

In [3]:
class GameState:
  def __init__(self, moves = []):
    self.children = []
    self.moves = moves
    self.winner = None
    self.terminal = False
    self.id = self.make_id()
  
  def make_id(self):
    if len(self.moves) != 0:
      return "-".join(["%d%d" % move for move in moves])
    else:
      return "root"
    
  def add_child(self, move):
    child = GameState([*self.moves, move])
    self.children.append(child)
    return child
  
  def get_child_state(self, move):
    return next((child for child in self.children if child.moves == [*self.moves, move]), None)
  
  def make_state_map(self, moves):
    state = {}
    player = 'X'
    for move in moves: 
      state[move] = player
      player = 'O' if player == 'X' else 'X'
    return state

def generate_games(game, parent_state):
  if len(game.state.moves) == 0:
    parent_state.terminal = True
    return
  
  if game.game_over(game.state):
    parent_state.winner = 'X' if game.state.to_move == 'O' else 'O'
    parent_state.terminal = True
    return
  
  if game.state.to_move == 'X':
    for move in game.state.moves:
      new_game = copy.deepcopy(game)
      new_game.state = new_game.result(move, new_game.state)
      generate_games(new_game, parent_state.add_child(move))
  else:
    new_game = copy.deepcopy(game)
    move = alphabeta_search(game)
    new_game.state = new_game.result(move, new_game.state)
    generate_games(new_game, parent_state.add_child(move))

   
t = TicTacToe(nrow=3, ncol=3, nwin=3)
root = GameState()
generate_games(t, root)

In [11]:
def node_str(node):
  state = node.make_state_map(node.moves)
  result = anchor % node.id
  
  use_links = len(node.children) != 0 and node.winner is None
  
  for row in range(1, 3+1):
    line = ""
    for col in range(1, 3+1):
      player_for_position = state.get((row, col), '_')
      
      display_player = player_map[player_for_position]
      
      if use_links and player_for_position == '_':
        o_move = node.get_child_state((row, col))
        if len(o_move.children) != 0:
          next_id = o_move.children[0].id
        else:
          next_id = o_move.id
        line += "[%s](#%s) " % (display_player, next_id)
      else:
        line += "%s " % display_player
    result += "%s<br/>" % line.rstrip()
  
  if node.winner is not None:
    result += win_message % node.winner
  elif len(node.moves) == 9:
    result += draw_message
  
  if node.terminal:
    result += "<br/>%s" % play_again
    
  result += "<br/><br/><br/>"
  return result

def bfs_display_v2(start):
  output = ""
  queue = [start]
  while len(queue) != 0:
    visit = queue.pop(0)
    queue.extend(visit.children)
    
    if len(visit.moves) % 2 == 0 or visit.terminal:
      output += "%s\n" % node_str(visit)
  return output
    
dfs = bfs_display_v2(root)

with io.open('README.md', 'w', encoding='utf-8') as f:
  f.write(header)
  f.write(dfs)
  f.write(footer)
print("Done")

Done
